In [1]:
import tensorflow as tf
import os
os.chdir("../")
from dataclasses import dataclass
from pathlib import Path

In [2]:
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories, save_json

In [3]:
@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data_path: Path
    params_validation_split: float
    params_image_size: list
    params_batch_size: int

## Configuration Manager

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ) -> None:
        """Inits ConfigurationManager.

        Args:
            config_file_path (Path, optional): Path of the config.yaml file.
                Defaults to the constant CONFIG_FILE_PATH.
            params_file_path (Path, optional): Path of the params.yaml file.
                Defaults to the constant PARAMS_FILE_PATH.
        """
        # Getting information in the config.yaml and params.yaml file
        self.config = read_yaml(yaml_file_path=config_file_path)
        self.params = read_yaml(yaml_file_path=params_file_path)
        
        # Creating the 'artifacts' directory
        create_directories(paths_of_directories=[self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            model_path=model_path,
            training_data_dir=training_data_dir,
            params_validation_split=self.params.VALIDATION_SPLIT,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
        )
        return evaluation_config

## Components

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig) -> None:
        """Inits Evaluation.

        Args:
            config (EvaluationConfig): The EvaluationConfig.
        """
        self.config = config
    
    def _val_generator(self):
        datagen_kwargs = dict(
            rescale=1./255,
            validation_split=self.config.params_validation_split,
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        
        val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagen_kwargs
        )
        
        self.validation_generator = val_datagen.flow_from_directory(
            directory=self.config.training_data_dir,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs,
        )
    
    def evaluation(self):
        self.model = self.load_model(path=self.config.model_path)
        self._val_generator()
        self.scores = self.model.evaluate(self.validation_generator)
    
    def save_scores(self):
        scores = {
            "loss": self.scores[0],
            "accuracy": self.scores[1]
        }
        save_json(
            path=Path("scores.json"),
            data=scores
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

In [ ]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e